<a href="https://colab.research.google.com/github/juanserrano90/codelatam/blob/main/Training/ViT/Augmented_Data_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://github.com/juanserrano90/codelatam/blob/main/Training/ViT/Augmented_Data_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/juanserrano90/codelatam.git

Cloning into 'codelatam'...
remote: Enumerating objects: 14321, done.
remote: Counting objects: 100% (864/864), done.
remote: Compressing objects: 100% (858/858), done.
remote: Total 14321 (delta 37), reused 801 (delta 6), pack-reused 13457 (from 1)
Receiving objects: 100% (14321/14321), 208.05 MiB | 17.29 MiB/s, done.
Resolving deltas: 100% (308/308), done.
Updating files: 100% (26535/26535), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read Data

In [3]:
# read augmented_images folder and create a list of images for each subfolder
import os
import numpy as np

image_lists = {}
root_dir = 'codelatam/Data/augmented_images_v2'

for subdir, dirs, files in os.walk(root_dir):
  for file in files:
    if file.endswith(('.jpg', '.jpeg', '.png')):
      subdir_name = os.path.basename(subdir)
      if subdir_name not in image_lists:
        image_lists[subdir_name] = []
      # image_lists[subdir_name].append(os.path.join(subdir, file))
      image_lists[subdir_name].append(file)

# Print the list of images for each subfolder
sum_img = 0
for subdir_name, image_list in image_lists.items():
  sum_img += len(image_list)
  print(f"Subfolder: {subdir_name}, Number of images: {len(image_list)}")
  print(image_list)  # Uncomment to print the full list of images
print(f"Total number of images: {sum_img}")


Subfolder: 0, Number of images: 2387
['sn2002eu_0_0_1.1_2074.png', 'sn2006bt_0_0_0.9_1825.png', 'sn2007sr_0_0_17.5_2335.png', 'sn2002de_0_0_1.8_213.png', 'sn94ae_0_0_40.1_1173.png', 'sn06N_0_0_-0.951_1278.png', 'sn2003hv_0_0_19.1_711.png', 'sn2007bd_0_0_8.4_647.png', 'sn2005hf_0_0_14.9_399.png', 'sn2002dj_0_0_219.6_786.png', 'sn1994M_0_0_1.6_791.png', 'sn2004gs_0_0_-3.2_1402.png', 'sn06or_0_0_-2.915_218.png', 'sn2003ch_0_0_0.8_1060.png', 'sn02er_0_0_15.5_1038.png', 'sn2003cg_0_0_-7.3_485.png', 'sn2005cf_0_0_5.2_1719.png', 'sn1995al_0_0_16.4_1005.png', 'sn2001ep_0_0_4.8_116.png', 'sn2001fh_0_0_-2.6_820.png', 'sn99dk_0_0_71.831_1393.png', 'sn2003hv_0_0_144.5_717.png', 'sn89B_0_0_31.4_442.png', 'sn2004gs_0_0_30.8_1410.png', 'sn2001ay_0_0_4.2_173.png', 'sn2007fb_0_0_10.6_1211.png', 'sn1994T_0_0_0.8_2289.png', 'sn2001gc_0_0_4.4_1348.png', 'sn2002jg_0_0_25.1_940.png', 'sn1997do_0_0_11.1_1448.png', 'sn2007fs_0_0_55.1_1399.png', 'sn2007bm_0_0_-9.3_571.png', 'sn2002cu_0_0_-6.3_48.png', 'sn2006c

In [4]:
def maping(a):
  subtype_to_category = {0:0, 1:1, 2:1, 3:1, 4:1, 5:1, 6:2, 7:2, 8:2, 9:2, 10:2, 11:2, 12:2, 13:2, 14:2, 15:2, 16:2}
  return subtype_to_category[a]

inv_dict_mapping_classes = {0:'Ia-norm', 1:'Ia-pec', 2:'Others'}

# Checking augmented data

In [5]:
# For each subfolder, separate the images that say COPY in their path from those that do not.
import os

def separate_images_with_copy(images_by_subfolder):
  """
  Separates images in each subfolder based on whether 'COPY' is in their path.

  Args:
    images_by_subfolder: A dictionary where keys are subfolder names and values are lists of image paths.

  Returns:
    A dictionary with the same subfolder keys, but values are now dictionaries
    containing 'with_copy' and 'without_copy' lists of images.
  """
  separated_images = {}
  for subfolder, image_list in images_by_subfolder.items():
    with_copy = []
    without_copy = []
    for image_path in image_list:
      if 'COPY' in image_path:
        if subfolder == '1':  # FIX 1: there is a problem with the SN labeling
          str_list = image_path.split("_")
          with_copy.append("_".join(str_list[:-3])) # remove the copy counter and the .png string
        else:
          str_list = image_path.split("_")
          if any(ele in str_list[1] for ele in ['6', '8', '10', '11', '16']): # FIX 2: There are categories labeled as copies (even though they are not)
            without_copy.append("_".join(str_list[:-2]))
          else:
            with_copy.append("_".join(str_list[:-2])) # remove the copy counter and the .png string
      else:
        str_list = image_path.split("_")
        without_copy.append("_".join(str_list[:-1]))
    separated_images[subfolder] = {'with_copy': with_copy, 'without_copy': without_copy}
  return separated_images

separated_images = separate_images_with_copy(image_lists)

for subfolder, image_dict in separated_images.items():
  print(f"Subfolder: {subfolder}")
  print(f"  Copied images: {len(image_dict['with_copy'])}")
  # Uncomment to print the list of images with 'COPY'
  print(f"    Image list: {image_dict['with_copy']}")
  print(f"  Original images: {len(image_dict['without_copy'])}")
  # Uncomment to print the list of images without 'COPY'
  print(f"    Image list: {image_dict['without_copy']}")


Subfolder: 0
  Copied images: 0
    Image list: []
  Original images: 2387
    Image list: ['sn2002eu_0_0_1.1', 'sn2006bt_0_0_0.9', 'sn2007sr_0_0_17.5', 'sn2002de_0_0_1.8', 'sn94ae_0_0_40.1', 'sn06N_0_0_-0.951', 'sn2003hv_0_0_19.1', 'sn2007bd_0_0_8.4', 'sn2005hf_0_0_14.9', 'sn2002dj_0_0_219.6', 'sn1994M_0_0_1.6', 'sn2004gs_0_0_-3.2', 'sn06or_0_0_-2.915', 'sn2003ch_0_0_0.8', 'sn02er_0_0_15.5', 'sn2003cg_0_0_-7.3', 'sn2005cf_0_0_5.2', 'sn1995al_0_0_16.4', 'sn2001ep_0_0_4.8', 'sn2001fh_0_0_-2.6', 'sn99dk_0_0_71.831', 'sn2003hv_0_0_144.5', 'sn89B_0_0_31.4', 'sn2004gs_0_0_30.8', 'sn2001ay_0_0_4.2', 'sn2007fb_0_0_10.6', 'sn1994T_0_0_0.8', 'sn2001gc_0_0_4.4', 'sn2002jg_0_0_25.1', 'sn1997do_0_0_11.1', 'sn2007fs_0_0_55.1', 'sn2007bm_0_0_-9.3', 'sn2002cu_0_0_-6.3', 'sn2006cp_0_0_7.3', 'sn2006ac_0_0_6.2', 'sn02er_0_0_-4.246', 'sn2002fk_0_0_87.6', 'sn04ey_0_0_-7.479', 'sn1997bq_0_0_40.9', 'sn2004dt_0_0_-9.9', 'sn1994D_0_0_42.6', 'sn2007fb_0_0_5.6', 'sn2003kf_0_0_13.4', 'sn2006ax_0_0_-6.4', 'sn02er

In [6]:
# for each 'without_copy' string, count how many strings are repeated in 'with_copy'
import collections

count_dict = {}

for subfolder, image_dict in separated_images.items():
  without_copy_list = image_dict['without_copy']
  with_copy_list = image_dict['with_copy']

  for image_without_copy in without_copy_list:
    count = with_copy_list.count(image_without_copy)
    if image_without_copy not in count_dict:
      count_dict[image_without_copy] = count
    else:
      count_dict[image_without_copy] += count

# create a pandas dataframe using count_dict
import pandas as pd

df = pd.DataFrame(list(count_dict.items()), columns=['Image', 'Copies'])
df['Category'] = df['Image'].str.split('_').str[1]
print(df)


                    Image  Copies Category
0        sn2002eu_0_0_1.1       0        0
1        sn2006bt_0_0_0.9       0        0
2       sn2007sr_0_0_17.5       0        0
3        sn2002de_0_0_1.8       0        0
4         sn94ae_0_0_40.1       0        0
...                   ...     ...      ...
4635     sn1998S_15_3_9.7       3       15
4636   sn1987A_16_3_989.4       0       16
4637  sn2004gt_10_2_22.49       0       10
4638     17ein_10_2_38.24       0       10
4639  sn2004et_13_3_168.8       0       13

[4640 rows x 3 columns]


In [7]:
SNsubtype_original = {i: 0 for i in range(0, 17)}
SNsubtype_copies = {i: 0 for i in range(0, 17)}
SNsubtype_total = {i: 0 for i in range(0, 17)}

for subfolder, image_dict in separated_images.items():
  without_copy_list = image_dict['without_copy']
  with_copy_list = image_dict['with_copy']

  for image_name in without_copy_list:
    if int(image_name.split('_')[1]) in SNsubtype_original:
      SNsubtype_original[int(image_name.split('_')[1])] += 1

  for image_name in with_copy_list:
    if int(image_name.split('_')[1]) in SNsubtype_copies:
      SNsubtype_copies[int(image_name.split('_')[1])] += 1

  for key,value in SNsubtype_total.items():
    SNsubtype_total[key] = SNsubtype_original[key] + SNsubtype_copies[key]

In [8]:
# convert SNsubtype_original, SNsubtype_copies and SNsubtype_total to a one pandas dataframe
#   where the columns labels are N° originals, N° copies and N° total respectively
#   and each row label is the key of the dictionary

import pandas as pd
id_to_subtype = {0: 'Ia-norm',
 1: 'Ia-91T',
 3: 'Ia-csm',
 2: 'Ia-91bg',
 6: 'Ib-norm',
 4: 'Iax',
 5: 'Ia-pec',
 10: 'Ic-norm',
 13: 'IIP',
 14: 'IIL',
 8: 'IIb',
 16: 'II-pec',
 11: 'Ic-broad',
 12: 'Ic-pec',
 15: 'IIn',
 7: 'Ibn',
 9: 'Ib-pec'}

# Create a list of dictionaries, where each dictionary represents a row
rows = []
for key in SNsubtype_original:
  row = {
      'N° originals': SNsubtype_original[key],
      'N° copies': SNsubtype_copies[key],
      'N° total': SNsubtype_total[key]
  }
  rows.append(row)

# Create the DataFrame from the list of dictionaries
df = pd.DataFrame(rows, index=SNsubtype_original.keys())
df.index.name = "ID"
df['Category'] = df.index.map(maping)
df['Subtype'] = df.index.map(id_to_subtype)
df = df[['Category', 'Subtype', 'N° originals', 'N° copies', 'N° total']]
print(df)

    Category   Subtype  N° originals  N° copies  N° total
ID                                                       
0          0   Ia-norm          2387          0      2387
1          1    Ia-91T           398         79       477
2          1   Ia-91bg           264        213       477
3          1    Ia-csm            30        447       477
4          1       Iax            68        409       477
5          1    Ia-pec           141        336       477
6          2   Ib-norm           217          0       217
7          2       Ibn            31        186       217
8          2       IIb           216          0       216
9          2    Ib-pec            15        202       217
10         2   Ic-norm           217          0       217
11         2  Ic-broad           217          0       217
12         2    Ic-pec            33        184       217
13         2       IIP           176         41       217
14         2       IIL            13        204       217
15         2  

Images per-class

In [9]:
# for each category, show the row sum of the column "N° total"
category_total = df.groupby('Category')['N° total'].sum()
print(category_total)

Category
0    2387
1    2385
2    2386
Name: N° total, dtype: int64


Unique SNs by class:

In [10]:
from collections import defaultdict

# Dictionary to store base name counts and their subtypes for each subfolder
alld = dict()
for subfolder, images in image_lists.items():
    alld[subfolder] = defaultdict(lambda: {'count': 0, 'subtype': set()})
    for image in images:
        base_name = image.split('_')[0]
        subtype = image.split('_')[1]
        alld[subfolder][base_name]['count'] += 1
        alld[subfolder][base_name]['subtype'].add(subtype)

# Print the results for each class
for i in range(3):
    unique_sn_count = len(alld[str(i)].keys())
    print(f'Class {i}: {unique_sn_count} unique SNs')

Class 0: 319 unique SNs
Class 1: 95 unique SNs
Class 2: 102 unique SNs


This is why we can't balance subtype in train/test for class 2
(each subtype has 217 images)

In [11]:
# Sort and print the top 10 base names by count for Class 2
class_2_data = alld['2']
sorted_class_2 = sorted(class_2_data.items(), key=lambda x: x[1]['count'], reverse=True)

print("Top 10 base names by count for Class 2:")
for base_name, data in sorted_class_2[:15]:
    print(f"{base_name}: Count = {data['count']}, Subtype = {data['subtype']}")

Top 10 base names by count for Class 2:
sn1987A: Count = 217, Subtype = {'16'}
sn1998S: Count = 195, Subtype = {'15'}
sn2004aw: Count = 169, Subtype = {'12'}
sn2007uy: Count = 157, Subtype = {'9'}
sn2006jc: Count = 141, Subtype = {'7'}
sn1980K: Count = 139, Subtype = {'14'}
sn1979C: Count = 78, Subtype = {'14'}
sn2005la: Count = 60, Subtype = {'9'}
sn1999em: Count = 56, Subtype = {'13'}
sn2004et: Count = 56, Subtype = {'13'}
sn2005cs: Count = 54, Subtype = {'13'}
sn2000er: Count = 50, Subtype = {'7'}
sn1993J: Count = 48, Subtype = {'8'}
sn2005ek: Count = 48, Subtype = {'12'}
sn2009jf: Count = 38, Subtype = {'6'}


In [12]:
# this SNs must be on train set (because they have too many images)
force_train = ['sn1987A', 'sn1998S', 'sn2004aw', 'sn2007uy', 'sn2006jc', 'sn1980K', 'sn1979C']

In [13]:
suma = 0
for i in force_train:
  suma = suma + alld['2'][i]['count']
print(suma)

1096


How many images in image_lists:

In [14]:
# count how many images are in image_lists
total_images = sum(len(images) for images in image_lists.values())
print(f"Total number of images: {total_images}")

Total number of images: 7158


# Holdout test data

In [15]:
import random
import collections

def create_train_test_splits(image_lists, train_ratio=0.85, test_ratio=0.15, force_train=['sn1987A', 'sn1998S', 'sn2004aw', 'sn2007uy', 'sn2006jc', 'sn1980K', 'sn1979C']):
    """
    Creates train and test splits for images in each subfolder,
    ensuring that images with the same base name are not repeated across splits,
    and that at least one base name for each subtype is included in the test set for subfolder 1.
    Additionally, for subfolder 2, ensures specific base names go into the train set.

    Args:
        image_lists: A dictionary where keys are subfolder names and values are lists of image paths.
        train_ratio: The proportion of images to include in the training set.
        test_ratio: The proportion of images to include in the test set.

    Returns:
        A dictionary with subfolder names as keys and values as dictionaries containing 'train' and 'test' lists of images.
    """
    splits = {}
    subtypes = {}
    # Subtype mappings for subfolder 1
    subtype_ranges = {
        "0": [0],             # Subfolder 0 has subtype 0
        "1": list(range(1, 6)),  # Subfolder 1 has subtypes 1 to 5
        "2": list(range(6, 17))  # Subfolder 2 has subtypes 6 to 16
    }

    # Base names to force into train set for subfolder 2 for train/test ['sn1987A', 'sn1998S', 'sn2004aw', 'sn2007uy', 'sn2006jc', 'sn1980K', 'sn1979C']

    for subfolder, image_list in image_lists.items():
        # Shuffle the image list to randomize
        random.shuffle(image_list)

        # Extract subtype information for each image
        subtype_dict = collections.defaultdict(list)
        for image_path in image_list:
            subtype = int(image_path.split('_')[1])  # Extract subtype as an integer
            subtype_dict[subtype].append(image_path)

        train_images = set()
        test_images = set()
        base_names = {}
        assigned_subtypes = set()

        if subfolder == "1":
            # Step 1: Ensure at least one base name for each subtype in the test set for subfolder 1
            for subtype in subtype_ranges[subfolder]:
                if subtype in subtype_dict:
                    paths = subtype_dict[subtype]
                    random.shuffle(paths)  # Shuffle to ensure randomness
                    for path in paths:
                        base_name = path.split('_')[0]
                        if base_name not in base_names:
                            base_names[base_name] = 'test'
                            test_images.add(path)
                            assigned_subtypes.add(subtype)
                            break  # Stop once we've added one base name for this subtype

        # Step 2: Distribute remaining images into train and test sets
        for image_path in image_list:
            base_name = image_path.split('_')[0]

            # Ensure specific base names go into train set for subfolder 2
            if subfolder == "2" and base_name in force_train:
                base_names[base_name] = 'train'
                train_images.add(image_path)
            else:
              if base_name not in base_names:
                  if random.random() < train_ratio:
                      base_names[base_name] = 'train'
                      train_images.add(image_path)
                  else:
                      base_names[base_name] = 'test'
                      test_images.add(image_path)
              else:
                  # Add to the respective split based on the assigned base name
                  if base_names[base_name] == 'train':
                      train_images.add(image_path)
                  elif base_names[base_name] == 'test':
                      test_images.add(image_path)

        # Collect the subtypes for train and test splits
        train_subtypes = {int(image.split('_')[1]) for image in train_images}
        test_subtypes = {int(image.split('_')[1]) for image in test_images}

        splits[subfolder] = {'train': list(train_images), 'test': list(test_images)}
        subtypes[subfolder] = {'train': train_subtypes, 'test': test_subtypes}

    return splits, subtypes

In [16]:
# # Keep trying splits until the ratios are within the tolerance.
# counter = 0
# while True:
#   splits, subtypes = create_train_test_splits(image_lists)
#   counter += 1
#   tolerance = 0.03
#   a = []
#   c = []
#   for i in range(3):
#     subtotal = len(splits[str(i)]['train'])+len(splits[str(i)]['test'])
#     a.append(round(len(splits[str(i)]['train'])/subtotal,2))
#     c.append(round(len(splits[str(i)]['test'])/subtotal,2))

#   # if abs(a[0]-0.85) < tolerance and  abs(c[0]-0.15) < tolerance and abs(a[1]-0.85) < tolerance and abs(c[1]-0.15) < tolerance and abs(a[2]-0.85) < tolerance and abs(c[2]-0.15) < tolerance and len(subtypes['2']['test']) == 8:
#   if abs(a[0]-0.85) < tolerance and  abs(c[0]-0.15) < tolerance and abs(a[1]-0.85) < tolerance and \
#      abs(c[1]-0.15) < tolerance and abs(a[2]-0.85) < tolerance and abs(c[2]-0.15) < tolerance and \
#      6 in subtypes['2']['test'] and 7 in subtypes['2']['test'] and 8 in subtypes['2']['test'] and \
#      10 in subtypes['2']['test'] and 11 in subtypes['2']['test'] and 13 in subtypes['2']['test'] and \
#      15 in subtypes['2']['test']:

#       good_split = splits
#       break

# # Print the number of images in each split for each subfolder
# print('iterations:', counter)
# for key, value in splits.items():
#   print(f"{key}: {value}")

In [17]:
# # check the number of images in each split
# total = 0
# print('images per split:')
# for i in range(3):
#   print(len(splits[str(i)]['train']), len(splits[str(i)]['test']))
#   total += len(splits[str(i)]['train']) + len(splits[str(i)]['test'])
# print('----------------')
# print('total images:', total)
# # proportions
# print('----------------')
# print('proportions:')
# for i in range(3):
#   subtotal = len(splits[str(i)]['train'])+len(splits[str(i)]['test'])
#   print(round(len(splits[str(i)]['train'])/subtotal,2), round(len(splits[str(i)]['test'])/subtotal,2))

#   splits[str(i)]['test']

In [18]:
# # Print the different subtypes in train and test for each subfolder
# for subfolder, subtype_info in subtypes.items():
#     print(f"Subfolder {subfolder}:")
#     print(f"  Train subtypes: {sorted(subtype_info['train'])}")
#     print(f"  Test subtypes: {sorted(subtype_info['test'])}")

In [19]:
import pickle
# save the dictionary to a file to recover the same split later (because it's random everytime)
# last split: 0.83, 0.86, 0.86, [6, 7, 8, 10, 11, 13, 15]

# with open('drive/MyDrive/lssds24/codelatam/saved_test_train_split.pkl', 'wb') as f:
#     pickle.dump(splits, f)

In [20]:
# load the saved split dictionary
# el archivo 'saved_splits.pkl' se puede cargar de mi google drive (juan)
# https://drive.google.com/drive/folders/1I9gk_JM-oFOD4xplydMrtrNrqHZeU0Jo?usp=sharing

# splits saved: [saved_test_train_split.pk]
with open('drive/MyDrive/lssds24/codelatam/saved_test_train_split.pkl', 'rb') as f:
    splits = pickle.load(f)

In [21]:
subts = []
class_ = '2'
for image in splits[class_]['train']:
  subt = image.split('_')[1]
  if subt not in subts:
    subts.append(subt)
print(f'Class {class_} subtypes on train:', subts)
subts = []
for image in splits[class_]['test']:
  subt = image.split('_')[1]
  if subt not in subts:
    subts.append(subt)
print(f'Class {class_} ssubtypes on test:', subts)

Class 2 subtypes on train: ['15', '9', '7', '11', '12', '6', '8', '14', '10', '16', '13']
Class 2 ssubtypes on test: ['6', '8', '7', '10', '15', '13', '11']


In [22]:
# check the number of images in each split
total = 0
print('images per split:')
for i in range(3):
  print(len(splits[str(i)]['train']), len(splits[str(i)]['test']))
  total += len(splits[str(i)]['train']) + len(splits[str(i)]['test'])
print('----------------')
print('total images:', total)
# proportions
print('----------------')
print('proportions:')
for i in range(3):
  subtotal = len(splits[str(i)]['train'])+len(splits[str(i)]['test'])
  print(round(len(splits[str(i)]['train'])/subtotal,2), round(len(splits[str(i)]['test'])/subtotal,2))

  splits[str(i)]['test']

images per split:
1984 403
2052 333
2057 329
----------------
total images: 7158
----------------
proportions:
0.83 0.17
0.86 0.14
0.86 0.14


In [23]:
# Subtypes count by split
subtype_counts_per_split = {}

for split_name in ['train', 'test']: # Assuming you have train and test splits
  subtype_counts_per_split[split_name] = {}
  for subfolder, data in splits.items():
    for image_name in data[split_name]:
      subtype = image_name.split('_')[1] # Extract subtype from filename
      if subtype not in subtype_counts_per_split[split_name]:
        subtype_counts_per_split[split_name][subtype] = 0
      subtype_counts_per_split[split_name][subtype] += 1

subtype_counts_per_split

{'train': {'5': 405,
  '1': 399,
  '2': 433,
  '3': 381,
  '4': 434,
  '0': 1984,
  '15': 195,
  '9': 217,
  '7': 167,
  '11': 190,
  '12': 217,
  '6': 114,
  '8': 157,
  '14': 217,
  '10': 165,
  '16': 217,
  '13': 201},
 'test': {'4': 43,
  '3': 96,
  '5': 72,
  '2': 44,
  '1': 78,
  '0': 403,
  '6': 103,
  '8': 59,
  '7': 50,
  '10': 52,
  '15': 22,
  '13': 16,
  '11': 27}}

In [24]:
# check if there are base names repeated in train and test in splits

def check_repeated_basenames(splits):
  """
  Checks for repeated base names in the train and test splits.

  Args:
    splits: A dictionary containing the train and test splits.

  Returns:
    A dictionary where keys are subfolder names and values are lists of
    repeated base names.
  """
  repeated_basenames = {}
  for subfolder, data in splits.items():
      train_basenames = set()
      test_basenames = set()
      repeated = []

      for image_name in data['train']:
          base_name = image_name.split('_')[0]
          train_basenames.add(base_name)

      for image_name in data['test']:
          base_name = image_name.split('_')[0]
          if base_name in train_basenames:
              repeated.append(base_name)
          test_basenames.add(base_name)

      if repeated:
        repeated_basenames[subfolder] = repeated
  return repeated_basenames


repeated_names = check_repeated_basenames(splits)

# Print the repeated base names for each subfolder
if repeated_names == {}:
  print('No repeated base names')
else:
  for subfolder, names in repeated_names.items():
    print(f"Subfolder: {subfolder}, Repeated base names: {names}")

No repeated base names


# Create validation splits

In [25]:
# # we can use the same function as before, but the input data should be only the train sets
# splits_train = {'0':splits['0']['train'], '1':splits['1']['train'] , '2':splits['2']['train']}

# force_train_val = ['sn1987A', 'sn1998S']

In [26]:
# # Keep trying splits until the ratios are within the tolerance.
# counter = 0
# while True:
#   splits_2, subtypes_2 = create_train_test_splits(splits_train, force_train=force_train_val)
#   counter += 1
#   tolerance = 0.021
#   a = []
#   c = []
#   for i in range(3):
#     subtotal = len(splits_2[str(i)]['train'])+len(splits_2[str(i)]['test'])
#     a.append(round(len(splits_2[str(i)]['train'])/subtotal,2))
#     c.append(round(len(splits_2[str(i)]['test'])/subtotal,2))

#     if abs(a[0]-0.85) < tolerance and  abs(c[0]-0.15) < tolerance and abs(a[1]-0.85) < tolerance and \
#     abs(c[1]-0.15) < tolerance and abs(a[2]-0.85) < tolerance and abs(c[2]-0.15) < tolerance:

#       good_split = splits_2
#       break

# # Print the number of images in each split for each subfolder
# print('iterations:', counter)
# for key, value in splits_2.items():
#   print(f"{key}: {value}")

In [27]:
# # # check the number of images in each split
# total = 0
# print('images per split:')
# for i in range(3):
#   print(len(splits_2[str(i)]['train']), len(splits_2[str(i)]['test']))
#   total += len(splits_2[str(i)]['train']) + len(splits_2[str(i)]['test'])
# print('----------------')
# print('total images:', total)
# # proportions
# print('----------------')
# print('proportions:')
# for i in range(3):
#   subtotal = len(splits_2[str(i)]['train'])+len(splits_2[str(i)]['test'])
#   print(round(len(splits_2[str(i)]['train'])/subtotal,2), round(len(splits_2[str(i)]['test'])/subtotal,2))

#   splits_2[str(i)]['test']

In [28]:
# # Print the different subtypes in train and test for each subfolder
# for subfolder, subtype_info in subtypes_2.items():
#     print(f"Subfolder {subfolder}:")
#     print(f"  Train subtypes: {sorted(subtype_info['train'])}")
#     print(f"  Test subtypes: {sorted(subtype_info['test'])}")

# Save 3 splits with same TEST, different train/val

In [29]:
# final_split = {'0':{'train':splits_2['0']['train'], 'val':splits_2['0']['test'], 'test':splits['0']['test']},
#                 '1':{'train':splits_2['1']['train'], 'val':splits_2['1']['test'], 'test':splits['1']['test']},
#                 '2':{'train':splits_2['2']['train'], 'val':splits_2['2']['test'], 'test':splits['2']['test']}}

# for i in range(3):
#   print(f'class {i}:')
#   print('train:', len(final_split[str(i)]['train']), 'val:', len(final_split[str(i)]['val']), 'test:', len(final_split[str(i)]['test']), '-- total:', len(final_split[str(i)]['train'])+len(final_split[str(i)]['val'])+len(final_split[str(i)]['test']))

In [30]:
# splits saved:
# saved_train_val_test_split1.pkl: Test subtypes: [6, 7, 8, 9, 10, 11]   # these are for fine-tuning
# saved_train_val_test_split2.pkl: Test subtypes: [6, 10, 11, 12, 13, 14]
# saved_train_val_test_split3.pkl: Test subtypes: [6, 8, 9, 10, 11, 13]

# with open('drive/MyDrive/lssds24/codelatam/saved_train_val_test_split3.pkl', 'wb') as f:
#     pickle.dump(final_split, f)

In [31]:
# load complete split
with open('drive/MyDrive/lssds24/codelatam/saved_train_val_test_split3.pkl', 'rb') as f:
    splits = pickle.load(f)

# Prepare data for modeling

In [32]:
import torch
from torchvision.io import read_image, ImageReadMode

train_data = {'id': [], 'pixel_values': [], 'y_true': []}
val_data = {'id': [], 'pixel_values': [], 'y_true': []}
test_data = {'id': [], 'pixel_values': [], 'y_true': []}

for subfolder, split_dict in splits.items():
  for split, image_list in split_dict.items():
    if split == 'train':
      for image_name in image_list:
        train_data['id'].append(image_name[:-4]) # remove the '.png' string
        train_data['pixel_values'].append(read_image(os.path.join(root_dir, subfolder, image_name), mode=ImageReadMode.RGB))
        train_data['y_true'].append(maping(int(image_name.split('_')[1])))
    elif split == 'val':
      for image_name in image_list:
        val_data['id'].append(image_name[:-4]) # remove the '.png' string
        val_data['pixel_values'].append(read_image(os.path.join(root_dir, subfolder, image_name), mode=ImageReadMode.RGB))
        val_data['y_true'].append(maping(int(image_name.split('_')[1])))
    elif split == 'test':
      for image_name in image_list:
        test_data['id'].append(image_name[:-4]) # remove the '.png' string
        test_data['pixel_values'].append(read_image(os.path.join(root_dir, subfolder, image_name), mode=ImageReadMode.RGB))
        test_data['y_true'].append(maping(int(image_name.split('_')[1])))


# stack 'pixel_values' list of tensors to a 4D tensor
train_data['pixel_values'] = torch.stack(train_data['pixel_values'])
train_data['y_true'] = torch.tensor(train_data['y_true'])

val_data['pixel_values'] = torch.stack(val_data['pixel_values'])
val_data['y_true'] = torch.tensor(val_data['y_true'])

test_data['pixel_values'] = torch.stack(test_data['pixel_values'])
test_data['y_true'] = torch.tensor(test_data['y_true'])

train_data['pixel_values'] = train_data['pixel_values'].permute(0, 2, 3, 1)
val_data['pixel_values'] = val_data['pixel_values'].permute(0, 2, 3, 1)
test_data['pixel_values'] = test_data['pixel_values'].permute(0, 2, 3, 1)

In [33]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.ids = data['id']
        self.pixel_values = data['pixel_values'].float()
        self.y_true = data['y_true'].long()

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        return {
            'id': self.ids[idx],
            'pixel_values': self.pixel_values[idx],
            'y_true': self.y_true[idx]
        }

train_dataset = CustomDataset(train_data)
val_dataset = CustomDataset(val_data)
test_dataset = CustomDataset(test_data)

# Finetuning ViT

In [34]:
!pip install -q transformers torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 418.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 951.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.7 MB/s eta 0:00:00


## Load pretrained model

In [35]:
import torch.nn as nn
from transformers import ViTImageProcessor, ViTModel

num_classes = len(torch.unique(train_data['y_true']))
num_classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pt_model_name = 'google/vit-base-patch32-224-in21k'
processor = ViTImageProcessor.from_pretrained(pt_model_name)
model = ViTModel.from_pretrained(pt_model_name).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/352M [00:00<?, ?B/s]

In [36]:
def load_model_and_classifier(wd, batch_size, lr, dropout):

    model = ViTModel.from_pretrained(pt_model_name).to(device)

    classifier = nn.Sequential(
        nn.Linear(model.config.hidden_size, 512),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(256, num_classes)
    ).to(device)

    return model, classifier

## Define train/validate/predict steps

In [37]:
import torch.nn.functional as F

def train_step(batch_data, model, processor, classifier, optimizer, device):
    model.train()
    y_true = batch_data['y_true'].to(device)

    inputs = processor(images=batch_data['pixel_values'], return_tensors="pt").to(device)
    outputs = model(**inputs)
    pooled_output = outputs.last_hidden_state[:, 0, :]

    logits = classifier(pooled_output)
    loss = F.cross_entropy(logits, y_true)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    y_pred = torch.argmax(logits, dim=-1)
    return loss.item(), y_pred, y_true


def validate_step(batch_data, model, processor, classifier, device):
    model.eval()
    with torch.no_grad():
        y_true = batch_data['y_true'].to(device)

        inputs = processor(images=batch_data['pixel_values'], return_tensors="pt").to(device)
        outputs = model(**inputs)
        pooled_output = outputs.last_hidden_state[:, 0, :]

        logits = classifier(pooled_output)
        loss = F.cross_entropy(logits, y_true)

        y_pred = torch.argmax(logits, dim=-1)
        return loss.item(), y_pred, y_true


def predict_step(batch_data, model, processor, classifier, device):
    model.eval()
    with torch.no_grad():
        y_true = batch_data['y_true'].to(device)

        inputs = processor(images=batch_data['pixel_values'], return_tensors="pt").to(device)
        outputs = model(**inputs)
        pooled_output = outputs.last_hidden_state[:, 0, :]

        logits = classifier(pooled_output)
        y_pred_prob = F.softmax(logits, dim=1)
        y_pred = torch.argmax(logits, dim=-1)

        return {
            'id': batch_data['id'],
            'y_pred': y_pred.cpu().numpy(),
            'y_pred_prob': y_pred_prob.cpu().numpy(),
            'y_true': y_true.cpu().numpy()
        }

## Train model

In [38]:
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [39]:
# # Define search ranges
learning_rates = [1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4]
dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
batch_sizes = [16, 32, 64]
weight_decays = [1e-4, 1e-3, 1e-2, 1e-1]

search_dict = {}

In [ ]:
from itertools import product
# Frist I search the best lr and batch size using a dropout of 0.3 and wd of 1e-4
# for three different splits of test/val/test
# best combination is batch_size=64 and lr=1e-5
# now search best dropout, choose 0.2
# now search best wd, choose 1e-2

# fine tuned params:
batch_size = 64
lr = 1e-5
dropout = 0.2
wd = 1e-2

plots_dict = {}
epochs = 100
patience = 7

# # hyperparameter grid search
# for wd in weight_decays:
print('------------------------------------------------------------------------------------------------')
print('------------------------------------------------------------------------------------------------')
print(f"Training with learning rate={lr}, dropout={dropout}, batch size={batch_size}, weight decay={wd}")

# this is for reproducibility ----------
seed = 10
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
generator = torch.Generator()
generator.manual_seed(seed)
# ---------------------------------------

# load model and classifier
model, classifier = load_model_and_classifier(wd, batch_size, lr, dropout)

# load datasets
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, generator=generator)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

params = list(model.parameters()) + list(classifier.parameters())
optimizer = AdamW(params, lr=lr, weight_decay=wd)

# save for ploting
t_loss = []
t_acc = []
t_f1 = []
v_loss = []
v_acc = []
v_f1 = []

best_val_loss = float('inf')
best_model_state = None
patience_counter = 0

for epoch in range(epochs):
  train_losses = []
  train_y_true = []
  train_y_pred = []

  for batch_data in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
    loss, y_pred, y_true = train_step(batch_data, model, processor, classifier, optimizer, device)
    train_losses.append(loss)
    train_y_true.extend(y_true.cpu().numpy())
    train_y_pred.extend(y_pred.cpu().numpy())

  train_loss = np.mean(train_losses)
  train_acc = accuracy_score(train_y_true, train_y_pred)
  train_f1 = f1_score(train_y_true, train_y_pred, average='macro')

  val_losses = []
  val_y_true = []
  val_y_pred = []

  for batch_data in val_dataloader:
    loss, y_pred, y_true = validate_step(batch_data, model, processor, classifier, device)
    val_losses.append(loss)
    val_y_true.extend(y_true.cpu().numpy())
    val_y_pred.extend(y_pred.cpu().numpy())

  val_loss = np.mean(val_losses)
  val_acc = accuracy_score(val_y_true, val_y_pred)
  val_f1 = f1_score(val_y_true, val_y_pred, average='macro')

  t_loss.append(train_loss)
  t_acc.append(train_acc)
  t_f1.append(train_f1)
  v_loss.append(val_loss)
  v_acc.append(val_acc)
  v_f1.append(val_f1)
  print(f"Epoch {epoch + 1}/{epochs}")
  print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}, Train F1: {train_f1:.4f}")
  print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}, Val F1: {val_f1:.4f}")

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    patience_counter = 0
    best_model_state = {
        'model': model.state_dict(),
        'classifier': classifier.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch
    }
  else:
    patience_counter += 1

  if patience_counter >= patience:
    print(f"Early stopping triggered. No improvement in validation loss for {patience} epochs.")
    break

plots_dict[f"lr_{lr}_dropout_{dropout}_batch_size={batch_size}_wd_{wd}"] = [t_loss, t_acc, t_f1, v_loss, v_acc, v_f1]

# search_dict[f"lr_{lr}_dropout_{dropout}_batch_size={batch_size}_wd_{wd}"] = {
#       'train_loss': t_loss,
#       'train_acc': t_acc,
#       'train_f1': t_f1,
#       'val_loss': v_loss,
#       'val_acc': v_acc,
#       'val_f1': v_f1}

# del train_dataloader, val_dataloader, test_dataloader, classifier, optimizer, model

torch.save(best_model_state, 'drive/MyDrive/lssds24/codelatam/v2_results/best_model_split3.pth')

In [41]:
# # # save search results
# with open('drive/MyDrive/lssds24/codelatam/v2_results/saved_search_results_3_wd_64.pkl', 'wb') as f:
#     pickle.dump(search_dict, f)

In [123]:
# # save dict of the last run for ploting
with open('drive/MyDrive/lssds24/codelatam/v2_results/plot_results_3.pkl', 'wb') as f:
    pickle.dump(plots_dict, f)

## Search Results

In [124]:
import matplotlib as mpl
cmap = mpl.colormaps['tab20']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re
from collections import defaultdict

# Load the search results from the pickle file
files = ['drive/MyDrive/lssds24/codelatam/v2_results/saved_search_results_1.pkl',
         'drive/MyDrive/lssds24/codelatam/v2_results/saved_search_results_2.pkl',
         'drive/MyDrive/lssds24/codelatam/v2_results/saved_search_results_3.pkl']

search_results = []
for file in files:
    with open(file, 'rb') as f:
        search_results.append(pickle.load(f))

# Combine dictionaries
all_dicts = [search_results[0], search_results[1], search_results[2]]
data = defaultdict(list)

# Extract relevant information
for d in all_dicts:
    for key, accuracies in d.items():
        # Extract learning rate and batch size
        match = re.search(r'lr_([\de.-]+)_.*_batch_size=(\d+)_.*', key)
        if match:
            lr = float(match.group(1))
            batch_size = int(match.group(2))
            # Corrected line: Access 'val_acc' and then find the maximum.
            max_accuracy = max(accuracies['val_acc'])
            data[batch_size].append((lr, max_accuracy))

# Process data for plotting
plot_data = {}
for batch_size, values in data.items():
    lr_to_accuracies = defaultdict(list)
    for lr, acc in values:
        lr_to_accuracies[lr].append(acc)
    means = {lr: np.mean(accs) for lr, accs in lr_to_accuracies.items()}
    stds = {lr: np.std(accs) for lr, accs in lr_to_accuracies.items()}
    plot_data[batch_size] = (list(means.keys()), list(means.values()), list(stds.values()))

# Plotting
# plt.figure(figsize=(10, 6))
for batch_size, (lrs, means, stds) in plot_data.items():
    plt.errorbar(lrs, means, yerr=stds, label=f'Batch Size {batch_size}', fmt='o-', capsize=6, alpha=0.7)

plt.xscale('log')
plt.xlabel('Learning Rate')
plt.ylabel('Max Validation Accuracy')
plt.title('Max Validation Accuracy vs Learning Rate')
plt.legend()
plt.grid(True)
# Set x-axis ticks and labels explicitly
plt.xticks(learning_rates, labels=[f'{lr:.1e}' for lr in learning_rates], rotation=45)

plt.tight_layout()  # Adjust layout to accommodate rotated labels
plt.savefig('drive/MyDrive/lssds24/codelatam/v2_results/max_val_acc_vs_lr_and_bs_3splits.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
for bs in batch_sizes:
  fig, axs = plt.subplots(1, 3, figsize=(15, 4))
  fig.suptitle(f'Batch size: {bs}')
  axs[0].plot(search_results[0][f'lr_1e-05_dropout_0.3_batch_size={bs}_wd_0.0001']['val_loss'], label='val')
  axs[0].plot(search_results[0][f'lr_1e-05_dropout_0.3_batch_size={bs}_wd_0.0001']['train_loss'], label='train')
  axs[0].legend()
  axs[1].plot(search_results[1][f'lr_1e-05_dropout_0.3_batch_size={bs}_wd_0.0001']['val_loss'])
  axs[1].plot(search_results[1][f'lr_1e-05_dropout_0.3_batch_size={bs}_wd_0.0001']['train_loss'])
  axs[2].plot(search_results[2][f'lr_1e-05_dropout_0.3_batch_size={bs}_wd_0.0001']['val_loss'])
  axs[2].plot(search_results[2][f'lr_1e-05_dropout_0.3_batch_size={bs}_wd_0.0001']['train_loss'])

In [ ]:
# Load the search results from the pickle file
files = ['drive/MyDrive/lssds24/codelatam/v2_results/saved_search_results_1_dropout_64.pkl',
         'drive/MyDrive/lssds24/codelatam/v2_results/saved_search_results_2_dropout_64.pkl',
         'drive/MyDrive/lssds24/codelatam/v2_results/saved_search_results_3_dropout_64.pkl']

search_results = []
for file in files:
    with open(file, 'rb') as f:
        search_results.append(pickle.load(f))

# Combine dictionaries
all_dicts = [search_results[0], search_results[1], search_results[2]]

# Initialize a dictionary to group accuracies by dropout
dp_accuracies = defaultdict(list)

# Extract dropout and maximum validation accuracies
for d in all_dicts:
    for key, accuracies in d.items():
        # Extract weight decay value from the key
        dp = float(key.split('_')[3])
        # Changed this line to get the max of the val_acc list
        max_acc = max(accuracies['val_acc'])
        dp_accuracies[dp].append(max_acc)

# Compute mean and standard deviation for each WD
dp_values = []
mean_acc = []
std_acc = []
lower_error = []
upper_error = []
mid_acc = []

for dp, accs in sorted(dp_accuracies.items()):
    dp_values.append(dp)
    mean_acc.append(np.mean(accs))
    std_acc.append(np.std(accs))
    mid = sum(accs) - max(accs) - min(accs)
    mid_acc.append(mid)
    lower_error.append(mid - min(accs))
    upper_error.append(max(accs) - mid)

# Plot
plt.errorbar(dp_values, mean_acc, yerr=std_acc, fmt='o-', capsize=5, label='Mean Accuracy')
plt.scatter(dp_values, mid_acc, label='Accuracies', marker='o', c='C1', zorder=90, alpha=0.4)
plt.scatter(dp_values, np.array(mid_acc)-np.array(lower_error), marker='o', c='C1', zorder=90, alpha=0.4)
plt.scatter(dp_values, np.array(mid_acc)+np.array(upper_error), marker='o', c='C1', zorder=90, alpha=0.4)


plt.xlabel('Dropout')
plt.ylim(0.81, 0.99)
plt.ylabel('Max Validation Accuracy')
plt.title('Max Validation Accuracy vs Dropout')
plt.grid(True)

# Set x-axis ticks and labels explicitly
plt.xticks(dp_values, labels=[f'{dp:.1f}' for dp in dp_values], rotation=45)

plt.legend()
plt.tight_layout()  # Adjust layout to accommodate rotated labels
plt.savefig('drive/MyDrive/lssds24/codelatam/v2_results/max_val_acc_vs_dropout_3splits.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# Load the search results from the pickle file
files = ['drive/MyDrive/lssds24/codelatam/v2_results/saved_search_results_1_wd_64.pkl',
         'drive/MyDrive/lssds24/codelatam/v2_results/saved_search_results_2_wd_64.pkl',
         'drive/MyDrive/lssds24/codelatam/v2_results/saved_search_results_3_wd_64.pkl']

search_results = []
for file in files:
    with open(file, 'rb') as f:
        search_results.append(pickle.load(f))

# Combine dictionaries
all_dicts = [search_results[0], search_results[1], search_results[2]]

# Initialize a dictionary to group accuracies by WD
wd_accuracies = defaultdict(list)

# Extract WD and maximum validation accuracies
for d in all_dicts:
    for key, accuracies in d.items():
        # Extract weight decay value from the key
        wd = float(key.split('_wd_')[1])
        # Changed this line to get the max of the val_acc list
        max_acc = max(accuracies['val_acc'])
        wd_accuracies[wd].append(max_acc)

# Compute mean and standard deviation for each WD
wd_values = []
mean_acc = []
std_acc = []
lower_error = []
upper_error = []
mid_acc = []

for wd, accs in sorted(wd_accuracies.items()):
    wd_values.append(wd)
    mean_acc.append(np.mean(accs))
    std_acc.append(np.std(accs))
    mid = sum(accs) - max(accs) - min(accs)
    mid_acc.append(mid)
    lower_error.append(mid - min(accs))
    upper_error.append(max(accs) - mid)

plt.errorbar(wd_values, mean_acc, yerr=std_acc, fmt='o-', capsize=5, label='Mean and Std', alpha=0.9, zorder=100)
plt.scatter(wd_values, mid_acc, label='Accuracies', marker='o', c='C1', zorder=90, alpha=0.4)
plt.scatter(wd_values, np.array(mid_acc)-np.array(lower_error), marker='o', c='C1', zorder=90, alpha=0.4)
plt.scatter(wd_values, np.array(mid_acc)+np.array(upper_error), marker='o', c='C1', zorder=90, alpha=0.4)

plt.xlabel('Weight Decay (WD)')
plt.xscale('log')
plt.ylabel('Max Validation Accuracy')
plt.title('Max Validation Accuracy vs Weight Decay')
plt.grid(True)
plt.ylim(0.84, 0.97)
plt.xticks(wd_values, labels=[f'{wd:.1e}' for wd in wd_values], rotation=45)

plt.legend()
plt.tight_layout()
plt.savefig('drive/MyDrive/lssds24/codelatam/v2_results/max_val_acc_vs_wd_3splits.png', dpi=200, bbox_inches='tight')
plt.show()

## Train results

In [125]:
# Load the plotting data of the last run from the pickle file
with open('drive/MyDrive/lssds24/codelatam/v2_results/plot_results_4.pkl', 'rb') as f:
    plots_dict = pickle.load(f)

for key, values in plots_dict.items():
  t_loss, t_acc, t_f1, v_loss, v_acc, v_f1 = values

In [ ]:
# Create a 1x3 grid of subplots
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot Loss
axes[0].plot(t_loss, label='Train Loss')
axes[0].plot(v_loss, label='Validation Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss Curves')
axes[0].legend()

# Plot Accuracy
axes[1].plot(t_acc, label='Train Accuracy')
axes[1].plot(v_acc, label='Validation Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy Curves')
axes[1].legend()

# Plot F1 Score
axes[2].plot(t_f1, label='Train F1 Score')
axes[2].plot(v_f1, label='Validation F1 Score')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('F1 Score')
axes[2].set_title('F1 Score Curves')
axes[2].legend()

# Adjust spacing between subplots
plt.tight_layout()
plt.savefig('drive/MyDrive/lssds24/codelatam/v2_results/training_split_3.png', dpi=200, bbox_inches='tight')
# Show the plot
plt.show()

In [ ]:
# Log loss
fig, axes = plt.subplots(figsize=(6, 4))

# Plot Loss
axes.plot(t_loss, label='Train Loss')
axes.plot(v_loss, label='Validation Loss')
axes.set_yscale('log')
axes.set_xlabel('Epoch')
axes.set_ylabel('Log-Loss')
axes.set_title('Loss Curves')
axes.legend()
plt.savefig('drive/MyDrive/lssds24/codelatam/v2_results/log-loss_split_3.png', dpi=200, bbox_inches='tight')

### Inference

In [128]:
# load best_model_**.pth, set to eval_mode

model.load_state_dict(torch.load('drive/MyDrive/lssds24/codelatam/v2_results/best_model_split3.pth')['model'])
classifier.load_state_dict(torch.load('drive/MyDrive/lssds24/codelatam/v2_results/best_model_split3.pth')['classifier'])
model.eval()
classifier.eval()

outputs = []
with torch.no_grad():
    for batch_data in test_dataloader:
        outputs.append(predict_step(batch_data, model, processor, classifier, device))

In [129]:
outputs[0].keys()

dict_keys(['id', 'y_pred', 'y_pred_prob', 'y_true'])

In [ ]:
keys = outputs[0].keys()
outputs_dict = {key: [] for key in keys}

for i in range(len(outputs)):
    for key in keys:
        outputs_dict[key].append(outputs[i][key])

outputs_dict = {key: np.concatenate(values) for key, values in outputs_dict.items()}
outputs_dict

In [131]:
outputs_dict['y_true'] = [inv_dict_mapping_classes[val] for val in outputs_dict['y_true']]
outputs_dict['y_pred'] = [inv_dict_mapping_classes[val] for val in outputs_dict['y_pred']]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(outputs_dict['y_true'], outputs_dict['y_pred'], digits=4))

### Confusion matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(outputs_dict['y_true'], outputs_dict['y_pred'])
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

class_names = inv_dict_mapping_classes.values()

plt.figure(figsize=(10, 7))
sns.heatmap(cm_normalized, annot=True, fmt='.3f', cmap='Blues', cbar=False,
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix (Normalized)')
#plt.savefig('drive/MyDrive/lssds24/codelatam/v2_results/confusion_matrix_split1.png', dpi=200, bbox_inches='tight')
plt.show()

In [95]:
subtype_mapping = {0: 'Ia-norm',
                   1: 'Ia-91T',
                   2: 'Ia-91bg',
                   3: 'Ia-csm',
                   4: 'Iax',
                   5: 'Ia-pec',
                   6: 'Ib-norm',
                   7: 'Ibn',
                   8: 'IIb',
                   9: 'Ib-pec',
                   10: 'Ic-norm',
                   11: 'Ic-broad',
                   12: 'Ic-pec',
                   13: 'IIP',
                   14: 'IIL',
                   15: 'IIn',
                   16: 'II-pec'}

### Missclassified images

In [ ]:
# identify all missclassified Images
missclassified_ia_norm_as_ia_pec = []
missclassified_ia_norm_as_others = []
missclassified_ia_pec_as_ia_norm = []
missclassified_ia_pec_as_others = []
missclassified_others_as_ia_norm = []
missclassified_others_as_ia_pec = []

for i in range(len(outputs_dict['y_true'])):
  if outputs_dict['y_true'][i] == 'Ia-norm' and outputs_dict['y_pred'][i] == 'Ia-pec':
    missclassified_ia_norm_as_ia_pec.append(outputs_dict['id'][i])
  elif outputs_dict['y_true'][i] == 'Ia-norm' and outputs_dict['y_pred'][i] == 'Others':
    missclassified_ia_norm_as_others.append(outputs_dict['id'][i])
  elif outputs_dict['y_true'][i] == 'Ia-pec' and outputs_dict['y_pred'][i] == 'Ia-norm':
    missclassified_ia_pec_as_ia_norm.append(outputs_dict['id'][i])
  elif outputs_dict['y_true'][i] == 'Ia-pec' and outputs_dict['y_pred'][i] == 'Others':
    missclassified_ia_pec_as_others.append(outputs_dict['id'][i])
  elif outputs_dict['y_true'][i] == 'Others' and outputs_dict['y_pred'][i] == 'Ia-norm':
    missclassified_others_as_ia_norm.append(outputs_dict['id'][i])
  elif outputs_dict['y_true'][i] == 'Others' and outputs_dict['y_pred'][i] == 'Ia-pec':
    missclassified_others_as_ia_pec.append(outputs_dict['id'][i])

print("Missclassified Ia-norm as Ia-pec images:")
print(missclassified_ia_norm_as_ia_pec)

print("Missclassified Ia-norm as Others images:")
print(missclassified_ia_norm_as_others)

In [ ]:
# make the plot in a nxn grid
import matplotlib.pyplot as plt

def plot_missclassified_images(image_ids, root_dir, subfolder, prediction, ncols=3):
  """Plots missclassified images from a list of image IDs in a grid.

  Args:
      image_ids: A list of image IDs.
      root_dir: The root directory where the images are stored.
      subfolder: The subfolder within the root directory where the images are stored.
      ncols: Number of columns in the grid.
  """

  nrows = (len(image_ids) + ncols - 1) // ncols  # Calculate the number of rows needed
  fig, axes = plt.subplots(nrows, ncols, figsize=(15, 3 * nrows))

  # Check if axes is 1-dimensional and convert to 2-dimensional if necessary
  if nrows == 1 or ncols == 1:
    axes = axes.reshape(nrows, ncols) # added this line to reshape axes to 2d if it's 1d

  for i, image_id in enumerate(image_ids):
    image_path = os.path.join(root_dir, subfolder, image_id + ".png")
    if os.path.exists(image_path):
      img = plt.imread(image_path)
      row = i // ncols
      col = i % ncols
      axes[row, col].imshow(img)
      axes[row, col].set_title(f"Missclassified Image: {image_id}\n Label: {inv_dict_mapping_classes[maping(int(image_id.split('_')[1]))]}, Subtype: {subtype_mapping[int(image_id.split('_')[1])]} \n Prediction: {prediction}")
      axes[row, col].axis('off')  # Turn off axis labels and ticks
  # Remove any empty subplots
  for i in range(len(image_ids), nrows * ncols):
    row = i // ncols
    col = i % ncols
    fig.delaxes(axes[row, col])

  plt.tight_layout()
  plt.show()

In [ ]:
# Plot the Ia-norm missclassified as ia-pec
plot_missclassified_images(missclassified_ia_norm_as_ia_pec, root_dir, "0", prediction='Ia-pec', ncols=3)

In [ ]:
# Plot the Ia-norm missclassified as other
plot_missclassified_images(missclassified_ia_norm_as_others, root_dir, "0", prediction='Others', ncols=2)

In [ ]:
#plot the Ia-pec missclassified as Ia-norm
plot_missclassified_images(missclassified_ia_pec_as_ia_norm, root_dir, "1", prediction='Ia-norm', ncols=3)

In [ ]:
plot_missclassified_images(missclassified_ia_pec_as_others, root_dir, "1", prediction='Others', ncols=3)

In [ ]:
# check how many Ia-91T are in test set, also save phases
Ia91t_in_test = []
phases = []
for i in splits['1']['test']:
  a = i.split('_')[1]
  if a == '1':
    Ia91t_in_test.append(i.split('_')[0])
    phases.append(float(i.split('_')[3]))
print(list(set(Ia91t_in_test)))
print('Ia-91T in test set:', len(Ia91t_in_test))
print('Ia-91T missclassified:', len(missclassified_ia_pec_as_ia_norm))

In [ ]:
miss_phases = []
for image_name in missclassified_ia_pec_as_ia_norm:
  miss_phases.append(float(image_name.split('_')[3]))

In [ ]:
plt.hist(phases, alpha=0.5, label='Ia-91T')
plt.hist(miss_phases, alpha=0.5, label='Ia-91T Missclassified')
plt.xlabel('Phase')
plt.ylabel('N')
plt.legend()
plt.savefig('drive/MyDrive/lssds24/codelatam/v2_results/Ia-91T-phases-histogram.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# I want to check if the Ia-norm's missclassified as Ia-pec are generally older SNs
# Need to get the year of the SN and plot an histogram of

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/juanserrano90/codelatam/refs/heads/main/Data/Redshift/sn_redshift.csv')

In [ ]:
years_miss = []
for image_name in missclassified_ia_norm_as_ia_pec:
  name = image_name.split('_')[0].lower()
  year = df[df['old_name'] == name]['year'].values[0]
  years_miss.append(year)
print(len(years_miss), len(missclassified_ia_norm_as_ia_pec))

years_ianorm = []
unique_ianorms = []
for image_name in image_lists['0']:
  name = image_name.split('_')[0].lower()
  if name not in unique_ianorms:
    unique_ianorms.append(name)
    year = df[df['old_name'] == name]['year'].values[0]
    years_ianorm.append(year)
print(len(years_ianorm), len(image_lists['0']))
print(len(unique_ianorms))

In [ ]:
print(np.array(years_ianorm).min(), np.array(years_ianorm).max())

In [ ]:
# Calculate medians and counts
median_ianorm = np.median(years_ianorm)
median_missclassified = np.median(years_miss)
count_ianorm = len(years_ianorm)
count_missclassified = len(years_miss)

# Print or use the results as needed
print(f"Median year for Ia-norm: {median_ianorm}")
print(f"Median year for missclassified Ia-norm: {median_missclassified}")
print(f"Total count of Ia-norm: {count_ianorm}")
print(f"Total count of missclassified Ia-norm: {count_missclassified}")

# Example of adding the medians to a plot (assuming the plot code from your example)

fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(years_ianorm, alpha=0.5, bins=np.arange(1981, 2010), align='left', edgecolor='black', label='Ia-norm')
ax.hist(years_miss, alpha=0.9, bins=np.arange(1981, 2010), align='left', edgecolor='black', label='Ia-norm Missclassified as Ia-pec')
ax.axvline(median_ianorm, color='blue', linestyle='dashed', linewidth=1, label=f'Ia-norm Median: {median_ianorm:.2f}')
ax.axvline(median_missclassified, color='red', linestyle='dashed', linewidth=1, label=f'Missclassified Median: {median_missclassified:.2f}')


plt.xlabel('Year')
plt.xticks(np.arange(1981, 2009), rotation=70)
plt.yticks(np.arange(0, 51, 5))
plt.ylabel('N')
plt.title('Year of discovery of Ia-norm SNs')
plt.legend()
plt.grid(True, alpha=0.4)
plt.savefig('drive/MyDrive/lssds24/codelatam/v2_results/Ia-norm-missclassidief-years-histogram_with_medians.png', dpi=200, bbox_inches='tight')
plt.show()